In [1]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader("qa_data.csv")
documents = loader.load()
texts = [doc.page_content for doc in documents]


In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('jeonseonjin/embedding_BAAI-bge-m3')
document_embeddings = model.encode(texts)


/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [10]:
import faiss
import numpy as np

# 질문-답변 FAISS 인덱스를 파일로 저장
index = faiss.IndexFlatL2(document_embeddings.shape[1])  
index.add(np.array(document_embeddings))
faiss.write_index(index, 'faiss_qna_index.bin')

# 질문
query = '질문 "지역화폐" 지역사랑상품권의 발행주체 부도시 한국은행의 보상 여부?'

# 질문에 대한 임베딩 생성
question_embedding = model.encode([query])

# 질문에 대해 FAISS에서 검색
D, I = index.search(np.array(question_embedding), k=1)
similarity = 1 / (1 + D[0][0])  # 유사도 계산

# 유사한 질문이 있을 경우 해당 답변 출력
print(f"질문에서 찾은 유사도: {similarity:.4f}")
print(f"답변: {texts[I[0][0]]}")  # texts 리스트에서 해당 인덱스의 텍스트를 찾아 출력


질문에서 찾은 유사도: 0.0040
답변: : 3
질문: 질문
"지역화폐" 지역사랑상품권의 발행주체 부도시 한국은행의 보상 여부?
답변: 답변
지역사랑상품권은 한국은행에서 발행한 법화(法貨)가 아니므로 발행주체의 부도시 한국은행은 이와 관련한 어떠한 보상도 하지 않습니다.


In [27]:
import faiss
import numpy as np

# 질문에 대한 답변을 제공하는 함수 정의
def get_answer(query, threshold=0.02):

    question_embedding = model.encode([query]) # 질문에 대한 임베딩 생성
    D, I = index.search(np.array(question_embedding), k=1) # FAISS에서 검색
    similarity = 100 / (1 + D[0][0])  # 유사도 계산 (거리가 0에 가까울수록 유사도는 1에 가까움)
    answer = texts[I[0][0]]  # 가장 유사한 질문에 해당하는 답변 반환

    # 유사도가 threshold 이상일 때만 답변 제공
    if similarity >= threshold:
        print(f"질문에서 찾은 유사도: {similarity:.4f}")
        return answer, True
    else:
        print(f"질문에서 유사한 결과를 찾지 못했습니다. 유사도: {similarity:.4f}")
        return answer, False


In [28]:
get_answer('전세계 외환시장의 거래규모')

질문에서 찾은 유사도: 0.1330


(': 19\n질문: 질문\n외환 매매기준율\n답변: 답변\n매매기준율이란 최근 거래일에 외국환중개회사를 통하여 거래가 이루어진 미달러화의 현물환율을 거래량으로 가중 평균하여 산출되는 시장평균환율을 말하며 각 외국환은행은 고객과 외환을 사고 팔 때 이를 기준으로 일정금액을 가감하여 대고객매매율을 정합니다. 즉, 각 외국환은행에서는 수수료 등을 포함하여 일정금액을 정한 후, 고객으로부터 외환을 살(팔) 때 매매기준율에서 이를 차감하여(더하여) 대고객매매율을 결정·고시하고 고객과의 외환거래 시 적용하고 있습니다.\n한편, 각 외국환은행이 적용하는 대고객매매율은 은행별로 다를 수 있습니다.',
 True)

In [30]:
import gradio as gr

# 질문에 대한 답변을 제공하는 함수 (get_answer 함수 수정)
def get_answer_bot(query, chatbot_history):
    # FAISS 인덱스에서 유사한 질문을 검색하고 답변을 찾는 로직
    # 이 부분은 기존 get_answer 함수의 로직을 사용
    question_embedding = model.encode([query])
    D, I = index.search(np.array(question_embedding), k=1)
    similarity = 100 / (1 + D[0][0])  # 유사도 계산

    if similarity >= 0.02:  # 유사도가 임계값 이상일 때 답변 반환
        answer = texts[I[0][0]]
    else:
        answer = "적절한 답변을 찾지 못했습니다."
    
    # 대화 기록 업데이트
    chatbot_history.append((query, answer))  # (질문, 답변)을 대화 기록에 추가
    return chatbot_history, ""

# Gradio Chatbot 인터페이스 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()  # 대화 기록을 표시하는 컴포넌트
    msg = gr.Textbox(label="질문 입력")  # 질문 입력을 위한 텍스트 박스
    clear = gr.Button("대화 기록 초기화")  # 대화 기록 초기화 버튼

    # 대화가 시작될 때 실행할 동작 정의
    msg.submit(get_answer_bot, inputs=[msg, chatbot], outputs=[chatbot, msg])  # 입력값을 처리 후 출력

    # 기록 초기화 버튼 동작 정의
    clear.click(lambda: [], None, chatbot, queue=False)  # 대화 기록을 초기화

# 앱 실행
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7864


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://c08cbb33456a7d329b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
질문 "지역화폐" 지역사랑상품권의 발행주체 부도시 한국은행의 보상 여부?